# Experiment 01

First Tests from the predictions.

In [1]:
import numpy as np
import pandas as pd
import math

import sys
sys.path.insert(1, 'C:\\Users\\Adrion\\Desktop\\TG\\DayTradeSystem\\Part1_Prediction')
import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')


def getPeriod2(df, begin, end, resetIndex = False):
    """
    Returns the df in the chosen interval
    
    Object begin: Start date forrmated as 'yyyy.mm.dd'.
    Object   end: End date 'yyyy.mm.dd'.

    returns a dataframe with the historic of the selected period
    """
    
    indexBegin = df[df['date']==begin].index[0]
    indexEnd = df[df['date']==end].tail(1).index[0]
    
    if (resetIndex):
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)].reset_index(drop=True)
    else: 
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)]

In [2]:
dfColumns = ['date','time','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']

TEST_BEGIN = '2021.04.06'
TEST_END = '2021.06.01'

TRAIN_BEGIN = '2020.03.31'
TRAIN_END = '2021.05.31'

TIMES = 10

In [3]:
df = pd.read_csv('../Data/PETR4_M15.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_M15.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

initialDf_PETR4 = getPeriod2(df, TEST_BEGIN, TEST_END)
initialDf_VALE3 = getPeriod2(df2, TEST_BEGIN, TEST_END)

In [4]:
initialDf_PETR4

,date,time,open,high,low,close
6197,2021.04.06,10:00:00,23.39,23.47,23.32,23.32
6198,2021.04.06,10:15:00,23.32,23.38,23.30,23.36
6199,2021.04.06,10:30:00,23.36,23.53,23.33,23.52
6200,2021.04.06,10:45:00,23.52,23.58,23.39,23.48
6201,2021.04.06,11:00:00,23.48,23.49,23.36,23.39
...,...,...,...,...,...,...
7311,2021.06.01,15:45:00,27.14,27.14,27.05,27.06
7312,2021.06.01,16:00:00,27.06,27.09,27.01,27.02
7313,2021.06.01,16:15:00,27.01,27.17,27.01,27.14
7314,2021.06.01,16:30:00,27.15,27.31,27.11,27.27


In [5]:
initialDf_VALE3

,date,time,open,high,low,close
6186,2021.04.06,10:00:00,103.80,104.35,103.52,103.77
6187,2021.04.06,10:15:00,103.77,103.80,102.92,103.23
6188,2021.04.06,10:30:00,103.23,104.25,103.00,104.23
6189,2021.04.06,10:45:00,104.23,104.56,103.70,104.03
6190,2021.04.06,11:00:00,104.03,104.12,103.15,103.34
...,...,...,...,...,...,...
7301,2021.06.01,15:45:00,113.84,113.95,113.61,113.72
7302,2021.06.01,16:00:00,113.72,114.00,113.70,113.99
7303,2021.06.01,16:15:00,113.99,114.00,113.90,113.96
7304,2021.06.01,16:30:00,113.97,114.00,113.88,113.88


In [6]:
pred_PETR4 = pd.read_pickle('../Data/PETR4_Pred.pkl')
pred_PETR4.head(5)

,high,low,high_pred,low_pred,date
559,23.58,23.13,23.566733,23.078467,2021.04.06
560,23.55,23.15,23.522933,23.064267,2021.04.07
561,23.26,22.61,23.355100,22.755300,2021.04.08
562,23.01,22.61,23.107800,22.629200,2021.04.09
563,23.58,23.01,23.488133,22.925933,2021.04.12


In [7]:
pred_VALE3 = pd.read_pickle('../Data/VALE3_Pred.pkl')
pred_VALE3.head(15)

,high,low,high_pred,low_pred,date
559,104.56,102.05,104.302233,101.745933,2021.04.06
560,105.32,102.15,104.931333,102.208100,2021.04.07
561,105.30,104.12,105.209333,103.450533,2021.04.08
562,104.66,102.94,104.909900,103.023500,2021.04.09
563,103.63,102.21,104.283567,102.585167,2021.04.12
564,104.29,102.85,104.736733,102.966533,2021.04.13
565,107.78,103.77,106.644333,103.458867,2021.04.14
566,109.88,107.55,109.097667,106.680967,2021.04.15
567,109.11,107.50,109.163233,107.258367,2021.04.16
568,109.88,107.64,109.677533,107.399533,2021.04.19


### Início do sistema

In [8]:
def getDay(df, day):
    return df[df['date']==day]

def buy(value, availableMoney, opr='normal'):
    lotValue = value*100
    quantity = math.floor(availableMoney/lotValue)
    
    amount = quantity*lotValue
    remainingMoney = availableMoney - amount
    
    return remainingMoney, quantity

def sell(value, amount, opr='normal'):
    if(opr == 'normal'):
        return value*amount*100
    else:
        return value*amount*100

In [16]:
def runStrategy(df, dailyDf, Money, stopLoss, std):
    size = len(df.index)

    Money = Money
    stopLoss = stopLoss

    orders = []

    for day in range(0,size):
        currentDay =  df.iloc[day].date

        lowPred = df.iloc[day].low_pred*(1+std)
        highPred = df.iloc[day].high_pred*(1-std)

        dayDf = getPeriod2(dailyDf, currentDay, currentDay)
        daySize = len(dayDf.index)

        op = 'none'
        quantity = 0
        trade = []

        for i in range(0,daySize):
            currentTime = dayDf.iloc[i]

            if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
                isStop = False
                if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
                aux = sell(currentTime.close, quantity)
                Money = Money + aux 
                trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
                isStop = False
                if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

                aux = sell(currentTime.close, quantity)
                Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
                trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif(op == 'none' and i < daySize-1 ):
                if(currentTime.close <= lowPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'buy'

                    trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]

                elif(currentTime.close >=  highPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'sell'

                    trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]
        print('eday '+str(day)+' money: '+str(Money))
    
    ordersDf = pd.DataFrame(orders)
    ordersDf.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
    return ordersDf

In [18]:
testPetr = runStrategy(pred_PETR4, initialDf_PETR4, 100000, 0.01, 0.0)
print('Accuracy: '+str(round(100*len(testPetr[testPetr['profit'] > 0].index)/len(testPetr.index), 2)))

testVale = runStrategy(pred_VALE3, initialDf_VALE3, 100000, 0.01, 0.0)
print('Accuracy: '+str(round(100*len(testVale[testVale['profit'] > 0].index)/len(testVale.index), 2)))

eday 0 money: 100000
eday 1 money: 100000
eday 2 money: 101056.0
eday 3 money: 101056.0
eday 4 money: 102736.0
eday 5 money: 102736.0
eday 6 money: 103510.0
eday 7 money: 103510.0
eday 8 money: 103960.0
eday 9 money: 104747.99999999999
eday 10 money: 104575.99999999999
eday 11 money: 104575.99999999999
eday 12 money: 104575.99999999999
eday 13 money: 104575.99999999999
eday 14 money: 108703.99999999999
eday 15 money: 108703.99999999999
eday 16 money: 111475.99999999999
eday 17 money: 111475.99999999999
eday 18 money: 111475.99999999999
eday 19 money: 111139.99999999999
eday 20 money: 111139.99999999999
eday 21 money: 111139.99999999999
eday 22 money: 111005.0
eday 23 money: 111005.0
eday 24 money: 110917.00000000001
eday 25 money: 110917.00000000001
eday 26 money: 110917.00000000001
eday 27 money: 110581.00000000001
eday 28 money: 110581.00000000001
eday 29 money: 110581.00000000001
eday 30 money: 110581.00000000001
eday 31 money: 111253.00000000001
eday 32 money: 111253.00000000001
ed

In [11]:
testPetr.head(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
0,2021.04.06,42,10:15:00,Sell,23.36,12:15:00,Buy,23.29,0.07,294.0,False,100294.0
1,2021.04.06,43,12:30:00,Buy,23.25,13:15:00,Sell,23.35,0.10,430.0,False,100724.0
2,2021.04.06,43,13:30:00,Buy,23.29,16:45:00,Sell,23.24,-0.05,-215.0,False,100509.0
3,2021.04.07,43,10:00:00,Buy,23.19,10:15:00,Sell,23.29,0.10,430.0,False,100939.0
4,2021.04.07,43,10:30:00,Sell,23.35,11:00:00,Buy,23.27,0.08,344.0,False,101283.0
5,2021.04.07,43,11:15:00,Sell,23.32,12:30:00,Buy,23.19,0.13,559.0,False,101842.0
6,2021.04.07,43,12:45:00,Buy,23.21,13:30:00,Sell,23.34,0.13,559.0,False,102401.0
7,2021.04.07,43,13:45:00,Sell,23.41,15:45:00,Buy,23.24,0.17,731.0,False,103132.0
8,2021.04.07,44,16:00:00,Buy,23.24,16:30:00,Sell,23.29,0.05,220.0,False,103352.0
9,2021.04.08,45,10:30:00,Buy,22.96,11:45:00,Sell,22.69,-0.27,-1215.0,True,102137.0


In [15]:
testVale.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
108,2021.05.25,13,12:00:00,Buy,108.29,15:45:00,Sell,107.15,-1.14,-1482.0,True,142458.0
109,2021.05.25,13,16:00:00,Buy,107.06,16:45:00,Sell,107.05,-0.01,-13.0,False,142445.0
110,2021.05.26,13,10:00:00,Buy,106.65,12:45:00,Sell,109.08,2.43,3159.0,False,145604.0
111,2021.05.26,13,13:00:00,Sell,109.21,16:45:00,Buy,110.20,-0.99,-1287.0,False,144317.0
112,2021.05.27,12,10:00:00,Sell,111.35,16:30:00,Buy,110.09,1.26,1512.0,False,145829.0
113,2021.05.28,13,10:15:00,Buy,110.26,13:30:00,Sell,111.34,1.08,1404.0,False,147233.0
114,2021.05.28,13,13:45:00,Sell,111.69,16:45:00,Buy,111.59,0.10,130.0,False,147363.0
115,2021.05.31,12,10:00:00,Sell,113.62,16:45:00,Buy,114.78,-1.16,-1392.0,True,145971.0
116,2021.06.01,12,10:00:00,Sell,117.54,14:15:00,Buy,114.43,3.11,3732.0,False,149703.0
117,2021.06.01,13,14:30:00,Buy,114.19,16:45:00,Sell,113.20,-0.99,-1287.0,False,148416.0


In [13]:
df = pred_VALE3
dailyDf = initialDf_VALE3
size = len(df.index)

Money = 100000
stopLoss = 0.01
LastTrade = ['None',10,100]

orders = []

for day in range(0,size):
    currentDay =  df.iloc[day].date

    lowPred = df.iloc[day].low_pred*1.01
    highPred = df.iloc[day].high_pred*0.99
    
    dayDf = getPeriod2(dailyDf, currentDay, currentDay)
    daySize = len(dayDf.index)
    
    op = 'none'
    quantity = 0
    trade = []
    
    print('Begining day '+str(day)+' money: '+str(Money))
    for i in range(0,daySize):
        currentTime = dayDf.iloc[i]
        
        if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
            isStop = False
            if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
            aux = sell(currentTime.close, quantity)
            Money = Money + aux 
            trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
            orders.append(trade)
  
            quantity = 0
            op = 'none'
        
        elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
            isStop = False
            if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

            aux = sell(currentTime.close, quantity)
            Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
            trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
            orders.append(trade)
            
            quantity = 0
            op = 'none'
        
        elif(op == 'none' and i < daySize-1 ):
            if(currentTime.close <= lowPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'buy'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]
                
            elif(currentTime.close >=  highPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'sell'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
test2.tail(60)

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']

print('Accuracy: '+str(round(100*len(test2[test2['profit'] > 0].index)/len(test2.index), 2)))
test2.head(10)

Begining day 0 money: 100000
Begining day 1 money: 100702.0
Begining day 2 money: 101521.0
Begining day 3 money: 102403.0
Begining day 4 money: 102079.0
Begining day 5 money: 103051.0
Begining day 6 money: 104470.00000000003
Begining day 7 money: 105018.00000000001
Begining day 8 money: 106512.00000000001
Begining day 9 money: 107340.00000000001
Begining day 10 money: 108634.0
Begining day 11 money: 109963.99999999997
Begining day 12 money: 110273.99999999999
Begining day 13 money: 110504.00000000001
Begining day 14 money: 111574.0
Begining day 15 money: 113704.00000000001
Begining day 16 money: 114474.0
Begining day 17 money: 116673.99999999997
Begining day 18 money: 119363.99999999994
Begining day 19 money: 118893.99999999994
Begining day 20 money: 119953.99999999994
Begining day 21 money: 120573.99999999994
Begining day 22 money: 120533.99999999994
Begining day 23 money: 122033.99999999994
Begining day 24 money: 125583.99999999996
Begining day 25 money: 127970.99999999994
Begining d

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
0,2021.04.06,9,10:00:00,Sell,103.77,16:00:00,Buy,102.71,1.06,954.0,False,100954.0
1,2021.04.06,9,16:15:00,Buy,102.33,16:45:00,Sell,102.05,-0.28,-252.0,False,100702.0
2,2021.04.07,9,10:00:00,Buy,102.76,13:30:00,Sell,103.95,1.19,1071.0,False,101773.0
3,2021.04.07,9,13:45:00,Sell,104.28,16:45:00,Buy,104.56,-0.28,-252.0,False,101521.0
4,2021.04.08,9,10:00:00,Sell,105.21,14:15:00,Buy,104.39,0.82,738.0,False,102259.0
5,2021.04.08,9,14:30:00,Sell,104.66,16:45:00,Buy,104.50,0.16,144.0,False,102403.0
6,2021.04.09,9,10:00:00,Buy,103.83,10:15:00,Sell,103.88,0.05,45.0,False,102448.0
7,2021.04.09,9,10:30:00,Sell,104.07,10:45:00,Buy,103.86,0.21,189.0,False,102637.0
8,2021.04.09,9,11:00:00,Buy,103.75,11:30:00,Sell,103.99,0.24,216.0,False,102853.0
9,2021.04.09,9,11:45:00,Buy,103.96,12:30:00,Sell,103.88,-0.08,-72.0,False,102781.0
